In [44]:
from IPython.display import display
import pandas as pd
import numpy as np

df=pd.read_csv("savant_data_2021_2023.csv")
df["game_date"] = pd.to_datetime(df["game_date"])
df["season"] = df["game_date"].dt.year  # Extract season year

In [45]:
# Aggregate the unique batters faced and count distinct games for each pitcher per season
unique_batters_per_season = (
    df.drop_duplicates(subset=["season", "batter", "game_pk", "pitcher"])
    .groupby(['pitcher', 'season'])
    .agg(
        Batters_Faced_by_pitcher=("batter", "size"),  # Count unique batters faced
        games_played_pitcher=("game_pk", "nunique")  # Count unique games
    )
    .reset_index()
)
# Count distinct at-bats and distinct games for each batter per season
batting_time_count = (
    df.drop_duplicates(subset=["season", "batter", "game_pk", "at_bat_number"])
    .groupby(["batter", "season"])
    .agg(
        Plate_Appearence_by_batter=("at_bat_number", "size"),  # Count unique at-bats
        games_played_batter=("game_pk", "nunique")       # Count unique games
    )
    .reset_index()
)

# Count of batters (pitches thrown) for each pitcher in a season
pitcher_pitches_thrown = (
    df.groupby(['pitcher', 'season'])
    .agg(
        pitches_thrown=("batter", "count")  # Count of batters faced
    )
    .reset_index()
)


# Count of pitchers (balls faced) for each batter in a season
batter_balls_faced = (
    df.groupby(['batter', 'season'])
    .agg(
        balls_faced=("pitcher", "count")  # Count of pitchers faced
    )
    .reset_index()
)

# Merge pitches thrown into unique batters faced per season
unique_batters_per_season = unique_batters_per_season.merge(
    pitcher_pitches_thrown,
    on=['pitcher', 'season'],
    how='left'
)

# Merge balls faced into distinct at-bats per season
batting_time_count = batting_time_count.merge(
    batter_balls_faced,
    on=['batter', 'season'],
    how='left'
)

# Strike zone count
df["is_in_strike_zone"] = df["zone"].isin([1, 2, 3, 4, 5, 6, 7, 8, 9])
strike_zone_count = df.groupby(["pitcher", "season"])["is_in_strike_zone"].sum()
unique_batters_per_season =unique_batters_per_season.merge(
    strike_zone_count,
    on=['pitcher', 'season'],
    how='left'
)


# strike out count
strikeout_count = df.groupby(["pitcher", "season"])["type"].apply(lambda x: (x == 'S').sum()).reset_index(name="strikeouts_by_pitcher")

unique_batters_per_season =unique_batters_per_season.merge(
    strikeout_count,
    on=['pitcher', 'season'],
    how='left'
)

# --- STEP 3: Identify Two-Way Players ---
# Get unique lists of batters and pitchers (per season)
batters = set(df["batter"].dropna().unique())
pitchers = set(df["pitcher"].dropna().unique())

# Identify players who appeared as both batters and pitchers
two_way_players = batters.intersection(pitchers)

# --- STEP 4: Merge Batting and Pitching Stats ---
# Rename columns for consistent merging
batting_time_count.rename(columns={"batter": "player"}, inplace=True)
unique_batters_per_season.rename(columns={"pitcher": "player"}, inplace=True)

# Merge batting and pitching stats by season
playing_time = pd.merge(
    batting_time_count,
    unique_batters_per_season,
    on=["season", "player"],
    how="outer"
).fillna(0)

playing_time["total_playing_time"] = playing_time["Batters_Faced_by_pitcher"] + playing_time["Plate_Appearence_by_batter"]

# Add a column to identify two-way players
playing_time["two_way_player"] = playing_time["player"].isin(two_way_players)

# # --- STEP 6: Handle Players with Single Roles ---
# # Optional: Add flags for role-specific players
# playing_time["is_batter_only"] = (playing_time["distinct_at_bats"] > 0) & (playing_time["distinct_at_pitch"] == 0)
# playing_time["is_pitcher_only"] = (playing_time["distinct_at_pitch"] > 0) & (playing_time["distinct_at_bats"] == 0)

display(playing_time.sort_values(by="total_playing_time", ascending=False))


,player,season,Plate_Appearence_by_batter,games_played_batter,balls_faced,Batters_Faced_by_pitcher,games_played_pitcher,pitches_thrown,is_in_strike_zone,strikeouts_by_pitcher,total_playing_time,two_way_player
1643,18396fcf5f98aac97ec6127f7924868d3ef7bd9e,2022,655.0,157.0,2546.0,253.0,28.0,2629.0,1352.0,1330.0,908.0,True
136,18396fcf5f98aac97ec6127f7924868d3ef7bd9e,2021,626.0,154.0,2594.0,210.0,23.0,2027.0,1044.0,973.0,836.0,True
3143,18396fcf5f98aac97ec6127f7924868d3ef7bd9e,2023,582.0,135.0,2321.0,203.0,23.0,2094.0,1088.0,1040.0,785.0,True
3465,514eeb6e6c17085fc1adabf03f1adca32318411f,2023,753.0,162.0,2799.0,0.0,0.0,0.0,0.0,0.0,753.0,False
4021,b4246bd7233f08e749626eb0b7647d295f2e37b7,2023,732.0,159.0,2782.0,0.0,0.0,0.0,0.0,0.0,732.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...
2279,852074cf280f7e76d76696a7a3e2ed58bb096c62,2022,1.0,1.0,4.0,0.0,0.0,0.0,0.0,0.0,1.0,False
704,776addf45775078d8c8694a7bd333845153f5529,2021,1.0,1.0,6.0,0.0,0.0,0.0,0.0,0.0,1.0,False
4350,eba4034f8b499f74e06bd5febb22a6934a8f7bac,2023,1.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,True
711,78576937fe7d731544ca5bd0f668e5f6b3dda9f3,2021,1.0,1.0,6.0,0.0,0.0,0.0,0.0,0.0,1.0,False


In [68]:
batters_df = pd.DataFrame(list(batters), columns=['batter'])
pitchers_df = pd.DataFrame(list(pitchers), columns=['pitcher'])

display(batters_df)

,batter
0,725357e7f2e42cfc4f08591df5caa9bcf5ff6280
1,c1a687ca25c227e4ad472453df39aad813b8cd5b
2,c9ce359e73fc58d171dff50208d0bd7bde1fef89
3,7b00d5dda854778e8d3ba783d4918faddae32575
4,efec6fbd26b9cfc55d11e70152a4eb986f75da43
...,...
1334,bcfe3b7e6b498b243aa5c5ddcb81d57e0176d976
1335,b505fcd3d7225538a30ecb5c4edab3e4b4b2c6bd
1336,c516e5ea6651f63cdfdd803bc2cce3965c3c822d
1337,cd008120919654d8408cbea3cf6b8908bf57dd02


In [69]:
df_submission = pd.read_csv("sample_submission.csv")

kaggle_df = df_submission.merge(
    pitchers_df,
    left_on=['PLAYER_ID'],
    right_on=['pitcher'],
    how='left'
)
kaggle_df = kaggle_df.merge(
    batters_df,
    left_on=['PLAYER_ID'],
    right_on=['batter'],
    how='left'
)

In [70]:
display(kaggle_df)

,PLAYER_ID,PLAYING_TIME,pitcher,batter
0,9a844e4d2ab1c3791241f7695bc86d1cbf40cba3,781,9a844e4d2ab1c3791241f7695bc86d1cbf40cba3,NaN
1,82cb9c0e72ca056b5de69fc76b30f69789740cbf,665,82cb9c0e72ca056b5de69fc76b30f69789740cbf,82cb9c0e72ca056b5de69fc76b30f69789740cbf
2,263f19273c6c4c12eceadb8824228fe1eac5e8a4,79,263f19273c6c4c12eceadb8824228fe1eac5e8a4,263f19273c6c4c12eceadb8824228fe1eac5e8a4
3,12dd288e43abf7690d8993f47d2bed8a024c8fef,117,12dd288e43abf7690d8993f47d2bed8a024c8fef,NaN
4,7d73305e306705436ac4f31f8149a297d95b0040,602,7d73305e306705436ac4f31f8149a297d95b0040,7d73305e306705436ac4f31f8149a297d95b0040
...,...,...,...,...
1144,1a35380236861a3e9242fc1790692a52ca977c92,395,NaN,1a35380236861a3e9242fc1790692a52ca977c92
1145,4624f24cd9ff75f49ed23c84b4f855ff2beb2ba2,136,NaN,4624f24cd9ff75f49ed23c84b4f855ff2beb2ba2
1146,837484931d4d9194973ae70dbc31984c986adcdb,390,NaN,837484931d4d9194973ae70dbc31984c986adcdb
1147,a4226052c7e0d04d4c9fdc737eada8c1d5d8e8ab,349,NaN,a4226052c7e0d04d4c9fdc737eada8c1d5d8e8ab


In [74]:
# Find rows where both pitcher and batter are NaN
rows_with_nan_pitcher_batter = kaggle_df[kaggle_df[['pitcher', 'batter']].isna().all(axis=1)]

# Display the rows
display(rows_with_nan_pitcher_batter)

,PLAYER_ID,PLAYING_TIME,pitcher,batter


In [12]:
# # Group by season and player type, then calculate descriptive statistics
# descriptive_stats = playing_time.groupby(
#     ['season', 'two_way_player', 'is_batter_only', 'is_pitcher_only']
# )['total_playing_time'].describe()

# # Display the statistics
# display(descriptive_stats)

count        mean  \
season two_way_player is_batter_only is_pitcher_only                      
2021   False          False          True             454.0  113.828194   
                      True           False            531.0  269.395480   
       True           False          False            452.0  189.986726   
                                     True               3.0  188.000000   
                      True           False             66.0  304.500000   
2022   False          False          True             493.0  119.758621   
                      True           False            564.0  269.739362   
       True           False          False             71.0  250.521127   
                                     True             307.0  178.908795   
                      True           False             58.0  246.068966   
2023   False          False          True             544.0  129.000000   
                      True           False            549.0  296.744991   
       True           False          False             65.0  215.723077   
                                     True             254.0  175.708661   
                      True           False             42.0  209.357143   

                                                             std   min  \
season two_way_player is_batter_only is_pitcher_only                     
2021   False          False          True              94.894604   3.0   
                      True           False            217.707641   1.0   
       True           False          False            112.065173   7.0   
                                     True             119.778963  50.0   
                      True           False            192.539646  11.0   
2022   False          False          True              91.756485   3.0   
                      True           False            219.634909   1.0   
       True           False          False            158.084142  19.0   
                                     True              91.129002   5.0   
                      True           False            197.458578   1.0   
2023   False          False          True              94.798555   2.0   
                      True           False            221.423251   1.0   
       True           False          False            155.310124  22.0   
                                     True              95.593020   4.0   
                      True           False            182.933924   1.0   

                                                         25%    50%     75%  \
season two_way_player is_batter_only is_pitcher_only                          
2021   False          False          True              28.25   85.0  201.00   
                      True           False             62.00  229.0  462.50   
       True           False          False             97.75  193.5  269.00   
                                     True             149.50  249.0  257.00   
                      True           False            142.00  302.0  451.50   
2022   False          False          True              33.00   97.0  203.00   
                      True           False             59.75  226.0  455.50   
       True           False          False            141.00  217.0  326.00   
                                     True              94.50  199.0  261.00   
                      True           False             83.00  198.0  390.75   
2023   False          False          True              45.00  107.0  213.25   
                      True           False             84.00  279.0  488.00   
       True           False          False             97.00  191.0  297.00   
                                     True              90.25  191.0  262.00   
                      True           False             65.25  148.5  348.75   

                                                        max  
season two_way_player is_batter_only is_pitcher_only         
2021   False          False          True          

In [4]:
# Count the number of players who are only batters, only pitchers, and two-way players
batter_only_count = playing_time["is_batter_only"].sum()
pitcher_only_count = playing_time["is_pitcher_only"].sum()
two_way_player_count = playing_time["two_way_player"].sum()

print("Count of batter-only players:", batter_only_count)
print("Count of pitcher-only players:", pitcher_only_count)
print("Count of two-way players:", two_way_player_count)

Count of batter-only players: 1810
Count of pitcher-only players: 2055
Count of two-way players: 1318


In [30]:
record_count = len(df)
print("Count of records in df:", record_count)

Count of records in df: 2136337


In [46]:
df['woba_value'].fillna(0, inplace=True)
df['woba_denom'].fillna(0, inplace=True)
df['babip_value'].fillna(0, inplace=True)
df['iso_value'].fillna(0, inplace=True)
df['estimated_ba_using_speedangle'].fillna(0, inplace=True)
df['estimated_woba_using_speedangle'].fillna(0, inplace=True)

C:\Users\siddh\AppData\Local\Temp\ipykernel_19700\908619352.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['woba_value'].fillna(0, inplace=True)
C:\Users\siddh\AppData\Local\Temp\ipykernel_19700\908619352.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when

### New Features

In [47]:
df['total_break'] = np.sqrt(df['pfx_x']**2 + df['pfx_z']**2)
df['velocity_drop'] = df['release_speed'] - df['effective_speed']
df['on_edge'] = ((abs(df['plate_x']) > 0.7) & (df['plate_z'] > df['sz_bot'] + 0.3) & (df['plate_z'] < df['sz_top'] - 0.3))
df['strike_zone_location'] = np.sqrt(df['plate_x']**2 + (df['plate_z'] - df['sz_bot'])**2)
df['Run_production_impact'] =  df['post_bat_score'] - df['bat_score']
mapping = {
    1: 'Weak',
    2: 'Topped',
    3: 'Under',
    4: 'Flare/Burner',
    5: 'Solid Contact',
    6: 'Barrel'
}

# Replace numeric values with names
df['launch_speed_angle_new'] = df['launch_speed_angle'].map(mapping)


In [48]:
import numpy as np
import pandas as pd

# Aggregating metrics for players per season
batter_season_metrics = (
    df.groupby(["batter", "season"])
    .agg(
        # Performance metrics
        total_woba_value=("woba_value", "sum"),  # Total wOBA contribution
        total_woba_denom=("woba_denom", "sum"),  # Total wOBA denominator
        total_babip_value=("babip_value", "sum"),  # Total BABIP contribution
        total_iso_value=("iso_value", "sum"),  # Total ISO contribution


        # Run production
        total_run_production=("Run_production_impact", "sum"),  # Total run production impact

        # Launch angle categories (counts of events by type)
        weak_contact_count_pitcher=("launch_speed_angle_new", lambda x: (x == "Weak").sum()),
        solid_contact_count_pitcher=("launch_speed_angle_new", lambda x: (x == "Solid Contact").sum()),
        barrel_count_pitcher=("launch_speed_angle_new", lambda x: (x == "Barrel").sum())
    )
    .reset_index()
)
pitcher_season_metrics = (
    df.groupby(["pitcher", "season"])
    .agg(
        total_babip_value=("babip_value", "sum"),  # Total BABIP contribution
        # Contact and pitch metrics
        avg_total_break=("total_break", "mean"),  # Average total break
        avg_velocity_drop=("velocity_drop", "mean"),  # Average velocity drop
        strike_zone_location_mean=("strike_zone_location", "mean"),  # Avg. strike zone distance
        pitches_on_edge=("on_edge", "sum"),  # Count of pitches on the edge of the strike zone


        # Launch angle categories (counts of events by type)
        weak_contact_count_batter=("launch_speed_angle_new", lambda x: (x == "Weak").sum()),
        topped_contact_count_batter=("launch_speed_angle_new", lambda x: (x == "Topped").sum()),
        under_contact_count_batter=("launch_speed_angle_new", lambda x: (x == "Under").sum()),
        flare_burner_contact_count_batter=("launch_speed_angle_new", lambda x: (x == "Flare/Burner").sum()),
        solid_contact_count_batter=("launch_speed_angle_new", lambda x: (x == "Solid Contact").sum()),
        barrel_count_batter=("launch_speed_angle_new", lambda x: (x == "Barrel").sum())
    )
    .reset_index()
)
# Display aggregated metrics per player and season
print(pitcher_season_metrics.head())


                                    pitcher  season  total_babip_value  \
0  007d7209f1d3287648e11f52c5a5148732f7a791    2021               13.0   
1  007d7209f1d3287648e11f52c5a5148732f7a791    2022               12.0   
2  007dcc596b82af90fd37f3413e98812b87a6b305    2021                1.0   
3  007dcc596b82af90fd37f3413e98812b87a6b305    2022                5.0   
4  007dcc596b82af90fd37f3413e98812b87a6b305    2023               42.0   

   avg_total_break  avg_velocity_drop  strike_zone_location_mean  \
0         1.313180          -1.103980                   1.461342   
1         1.278204          12.614644                   1.504396   
2         1.573777           0.137037                   1.314674   
3         1.587169           0.566013                   1.328496   
4         1.538237           0.704103                   1.380921   

   pitches_on_edge  weak_contact_count_batter  topped_contact_count_batter  \
0               29                          1                       

In [49]:
final_df = playing_time.merge(
    batter_season_metrics,
    left_on=['player', 'season'],
    right_on=['batter', 'season'],
    how='left'
)

display(final_df)

,player,season,Plate_Appearence_by_batter,games_played_batter,balls_faced,Batters_Faced_by_pitcher,games_played_pitcher,pitches_thrown,is_in_strike_zone,strikeouts_by_pitcher,...,two_way_player,batter,total_woba_value,total_woba_denom,total_babip_value,total_iso_value,total_run_production,weak_contact_count_pitcher,solid_contact_count_pitcher,barrel_count_pitcher
0,003af1e4636109b822c9acfa703cb517c46d89fc,2021,163.0,44.0,618.0,0.0,0.0,0.0,0.0,0.0,...,False,003af1e4636109b822c9acfa703cb517c46d89fc,49.10,162.0,33.0,20.0,14.0,8.0,3.0,7.0
1,007d7209f1d3287648e11f52c5a5148732f7a791,2021,0.0,0.0,0.0,55.0,10.0,201.0,110.0,94.0,...,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,007dcc596b82af90fd37f3413e98812b87a6b305,2021,0.0,0.0,0.0,6.0,2.0,27.0,17.0,17.0,...,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,00e99b6a19161174f9f6520a80783c70236d8403,2021,36.0,14.0,136.0,0.0,0.0,0.0,0.0,0.0,...,False,00e99b6a19161174f9f6520a80783c70236d8403,12.35,36.0,8.0,4.0,2.0,1.0,0.0,2.0
4,01062805175a65bbfb5e2024853137219296e26c,2021,559.0,137.0,1877.0,0.0,0.0,0.0,0.0,0.0,...,True,01062805175a65bbfb5e2024853137219296e26c,186.35,558.0,133.0,61.0,63.0,15.0,32.0,19.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4448,fe6eac0778f476c3f1ffe6a1fd637f077573e077,2023,531.0,119.0,2205.0,0.0,0.0,0.0,0.0,0.0,...,False,fe6eac0778f476c3f1ffe6a1fd637f077573e077,178.75,527.0,94.0,74.0,64.0,10.0,27.0,27.0
4449,fe80da2e964fa0782b5d30725105f59444a90261,2023,200.0,50.0,761.0,0.0,0.0,0.0,0.0,0.0,...,False,fe80da2e964fa0782b5d30725105f59444a90261,68.10,199.0,29.0,48.0,29.0,1.0,9.0,21.0
4450,ff3b7d21e60c24b6e926e3ffc3fbeb78cc9e4057,2023,0.0,0.0,0.0,258.0,68.0,929.0,509.0,442.0,...,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4451,ffa57541bf6d7030fdf7206f4aa0141a2c918647,2023,0.0,0.0,0.0,217.0,24.0,2100.0,1045.0,989.0,...,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
final_df = final_df.merge(
    pitcher_season_metrics,
    left_on=['player', 'season'],
    right_on=['pitcher', 'season'],
    how='left'
)

display(final_df)

,player,season,Plate_Appearence_by_batter,games_played_batter,balls_faced,Batters_Faced_by_pitcher,games_played_pitcher,pitches_thrown,is_in_strike_zone,strikeouts_by_pitcher,...,avg_total_break,avg_velocity_drop,strike_zone_location_mean,pitches_on_edge,weak_contact_count_batter,topped_contact_count_batter,under_contact_count_batter,flare_burner_contact_count_batter,solid_contact_count_batter,barrel_count_batter
0,003af1e4636109b822c9acfa703cb517c46d89fc,2021,163.0,44.0,618.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,007d7209f1d3287648e11f52c5a5148732f7a791,2021,0.0,0.0,0.0,55.0,10.0,201.0,110.0,94.0,...,1.313180,-1.103980,1.461342,29.0,1.0,10.0,12.0,12.0,0.0,4.0
2,007dcc596b82af90fd37f3413e98812b87a6b305,2021,0.0,0.0,0.0,6.0,2.0,27.0,17.0,17.0,...,1.573777,0.137037,1.314674,8.0,0.0,3.0,0.0,0.0,0.0,0.0
3,00e99b6a19161174f9f6520a80783c70236d8403,2021,36.0,14.0,136.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,01062805175a65bbfb5e2024853137219296e26c,2021,559.0,137.0,1877.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4448,fe6eac0778f476c3f1ffe6a1fd637f077573e077,2023,531.0,119.0,2205.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4449,fe80da2e964fa0782b5d30725105f59444a90261,2023,200.0,50.0,761.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4450,ff3b7d21e60c24b6e926e3ffc3fbeb78cc9e4057,2023,0.0,0.0,0.0,258.0,68.0,929.0,509.0,442.0,...,1.125692,0.885129,1.431944,159.0,13.0,94.0,27.0,46.0,3.0,9.0
4451,ffa57541bf6d7030fdf7206f4aa0141a2c918647,2023,0.0,0.0,0.0,217.0,24.0,2100.0,1045.0,989.0,...,1.390118,-0.699333,1.289189,350.0,19.0,87.0,143.0,85.0,27.0,28.0


In [76]:
unique_players = final_df['player'].unique()
unique_players_df = pd.DataFrame(list(unique_players), columns=['player'])
kaggle_df = df_submission.merge(
    unique_players_df,
    left_on=['PLAYER_ID'],
    right_on=['player'],
    how='inner'
)
display(kaggle_df)

,PLAYER_ID,PLAYING_TIME,player
0,9a844e4d2ab1c3791241f7695bc86d1cbf40cba3,781,9a844e4d2ab1c3791241f7695bc86d1cbf40cba3
1,82cb9c0e72ca056b5de69fc76b30f69789740cbf,665,82cb9c0e72ca056b5de69fc76b30f69789740cbf
2,263f19273c6c4c12eceadb8824228fe1eac5e8a4,79,263f19273c6c4c12eceadb8824228fe1eac5e8a4
3,12dd288e43abf7690d8993f47d2bed8a024c8fef,117,12dd288e43abf7690d8993f47d2bed8a024c8fef
4,7d73305e306705436ac4f31f8149a297d95b0040,602,7d73305e306705436ac4f31f8149a297d95b0040
...,...,...,...
1144,1a35380236861a3e9242fc1790692a52ca977c92,395,1a35380236861a3e9242fc1790692a52ca977c92
1145,4624f24cd9ff75f49ed23c84b4f855ff2beb2ba2,136,4624f24cd9ff75f49ed23c84b4f855ff2beb2ba2
1146,837484931d4d9194973ae70dbc31984c986adcdb,390,837484931d4d9194973ae70dbc31984c986adcdb
1147,a4226052c7e0d04d4c9fdc737eada8c1d5d8e8ab,349,a4226052c7e0d04d4c9fdc737eada8c1d5d8e8ab


In [51]:
# filtered_batting_summary = playing_time[
#     (playing_time["player"] == '0014c193005b425aaad55358686fb0dd1a4a0755') & 
#     (playing_time["season"] == 2023)
# ]
# display(filtered_batting_summary)    

final_df['two_way_player'] = final_df['two_way_player'].astype(int)


In [34]:
print(final_df.columns)

Index(['player', 'season', 'Plate_Appearence_by_batter', 'games_played_batter',
       'balls_faced', 'Batters_Faced_by_pitcher', 'games_played_pitcher',
       'pitches_thrown', 'is_in_strike_zone', 'strikeouts_by_pitcher',
       'total_playing_time', 'two_way_player', 'batter', 'total_woba_value',
       'total_woba_denom', 'total_babip_value_x', 'total_iso_value',
       'total_run_production', 'weak_contact_count_pitcher',
       'solid_contact_count_pitcher', 'barrel_count_pitcher', 'pitcher',
       'total_babip_value_y', 'avg_total_break', 'avg_velocity_drop',
       'strike_zone_location_mean', 'pitches_on_edge',
       'weak_contact_count_batter', 'topped_contact_count_batter',
       'under_contact_count_batter', 'flare_burner_contact_count_batter',
       'solid_contact_count_batter', 'barrel_count_batter'],
      dtype='object')


In [57]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt

# Handle missing values
final_df.fillna(0, inplace=True)

# **🔹 Rolling Features (3-year average)**
final_df.sort_values(by=['player', 'season'], inplace=True)
rolling_features = [ 'games_played_batter',
       'balls_faced', 'games_played_pitcher',
       'pitches_thrown', 'is_in_strike_zone', 'strikeouts_by_pitcher',
       'total_playing_time', 'two_way_player', 'total_woba_value',
       'total_woba_denom', 'total_babip_value_x', 'total_iso_value',
       'total_run_production', 'weak_contact_count_pitcher',
       'solid_contact_count_pitcher', 'barrel_count_pitcher',
       'total_babip_value_y', 'avg_total_break', 'avg_velocity_drop',
       'strike_zone_location_mean', 'pitches_on_edge',
       'weak_contact_count_batter', 'topped_contact_count_batter',
       'under_contact_count_batter', 'flare_burner_contact_count_batter',
       'solid_contact_count_batter', 'barrel_count_batter']
for feature in rolling_features:
    final_df[f'rolling_{feature}'] = final_df.groupby('player')[feature].transform(lambda x: x.rolling(3, min_periods=1).mean())

# Train on 2021-2023 data
train_data = final_df[final_df['season'] < 2024]

# Define Features (X) and Target (y)
X = train_data.drop(columns=['player', 'season', 'total_playing_time','Plate_Appearence_by_batter','Batters_Faced_by_pitcher','batter','pitcher'])
y = train_data['total_playing_time']

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# **🔹 Train Multiple Models**
models = {
    "XGBoost": XGBRegressor(n_estimators=200, learning_rate=0.1, max_depth=5, enable_categorical=True),
    "RandomForest": RandomForestRegressor(n_estimators=200, max_depth=5, random_state=42),
    "LinearRegression": LinearRegression()
}

results = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))  # Compute RMSE
    
    results[name] = {"MAE": mae, "RMSE": rmse}  # Store both MAE & RMSE
    
    print(f"{name} - MAE: {mae:.2f}, RMSE: {rmse:.2f}")

# **🔹 Select Best Model**
best_model = min(results, key=lambda x: results[x]["RMSE"])  # Select model with lowest RMSE
final_model = models[best_model]

# **🔹 Feature Importance (XGBoost & RandomForest only)**
if best_model in ["XGBoost", "RandomForest"]:
    feature_importance = final_model.feature_importances_
    sorted_idx = np.argsort(feature_importance)[::-1]

    plt.figure(figsize=(10, 5))
    plt.barh(np.array(X.columns)[sorted_idx], feature_importance[sorted_idx])
    plt.xlabel("Feature Importance")
    plt.ylabel("Features")
    plt.title(f"Feature Importance - {best_model}")
    plt.show()

# **🔹 Predict 2024**
predict_data = final_df[final_df['season'] == 2023].copy()
predict_data['season'] = 2024  
X_predict = predict_data.drop(columns=['player', 'season', 'total_playing_time','Plate_Appearence_by_batter','Batters_Faced_by_pitcher','batter','pitcher'])
predict_data['predicted_playing_time'] = np.ceil(final_model.predict(X_predict))

# Drop duplicates and keep max playing time
predict_data = predict_data.sort_values(by=['player', 'predicted_playing_time'], ascending=False)
predict_data = predict_data.drop_duplicates(subset=['player'], keep='first')

# Save predictions
predict_data[['player', 'predicted_playing_time']].to_csv("predictions_2024.csv", index=False)

print(f"✅ 2024 Predictions using {best_model} Saved!")


XGBoost - MAE: 4.57, RMSE: 6.58
RandomForest - MAE: 15.92, RMSE: 26.02
LinearRegression - MAE: 2.23, RMSE: 4.64
✅ 2024 Predictions using LinearRegression Saved!


In [59]:
df_submission = pd.read_csv("sample_submission.csv")

kaggle_df = predict_data.merge(
    df_submission,
    left_on=['player'],
    right_on=['PLAYER_ID'],
    how='left'
)

display(kaggle_df) 

,player,season,Plate_Appearence_by_batter,games_played_batter,balls_faced,Batters_Faced_by_pitcher,games_played_pitcher,pitches_thrown,is_in_strike_zone,strikeouts_by_pitcher,...,rolling_pitches_on_edge,rolling_weak_contact_count_batter,rolling_topped_contact_count_batter,rolling_under_contact_count_batter,rolling_flare_burner_contact_count_batter,rolling_solid_contact_count_batter,rolling_barrel_count_batter,predicted_playing_time,PLAYER_ID,PLAYING_TIME
0,ffc9b6a7a663bc322e74c70a5ed8756787a577d0,2024,0.0,0.0,0.0,237.0,60.0,969.0,474.0,463.0,...,196.333333,10.0,44.333333,40.000000,37.666667,11.000000,11.333333,234.0,NaN,NaN
1,ffa57541bf6d7030fdf7206f4aa0141a2c918647,2024,0.0,0.0,0.0,217.0,24.0,2100.0,1045.0,989.0,...,348.000000,19.0,84.000000,129.500000,80.500000,24.500000,30.500000,219.0,ffa57541bf6d7030fdf7206f4aa0141a2c918647,520.0
2,ff3b7d21e60c24b6e926e3ffc3fbeb78cc9e4057,2024,0.0,0.0,0.0,258.0,68.0,929.0,509.0,442.0,...,126.000000,8.0,71.333333,21.000000,34.333333,3.666667,6.000000,272.0,ff3b7d21e60c24b6e926e3ffc3fbeb78cc9e4057,202.0
3,fe80da2e964fa0782b5d30725105f59444a90261,2024,200.0,50.0,761.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,200.0,fe80da2e964fa0782b5d30725105f59444a90261,108.0
4,fe6eac0778f476c3f1ffe6a1fd637f077573e077,2024,531.0,119.0,2205.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,529.0,fe6eac0778f476c3f1ffe6a1fd637f077573e077,532.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1449,0084c15ee1d82fb5b793e1ff130f46651dd13e17,2024,60.0,24.0,246.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,61.0,0084c15ee1d82fb5b793e1ff130f46651dd13e17,41.0
1450,007dcc596b82af90fd37f3413e98812b87a6b305,2024,0.0,0.0,0.0,213.0,42.0,853.0,455.0,414.0,...,69.333333,3.0,21.000000,12.666667,13.333333,3.333333,3.000000,195.0,NaN,NaN
1451,0043ac96d4fde6fcfd5a841b8d902661e69a6009,2024,44.0,12.0,200.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,45.0,0043ac96d4fde6fcfd5a841b8d902661e69a6009,33.0
1452,003af1e4636109b822c9acfa703cb517c46d89fc,2024,91.0,28.0,327.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,91.0,003af1e4636109b822c9acfa703cb517c46d89fc,131.0
